In [4]:
import serial
import numpy as np
import matplotlib.pyplot as plt
import time
import os
from scipy.signal import find_peaks, peak_prominences
%matplotlib notebook


def read_arduino(ser,inputBufferSize):
#    data = ser.readline(inputBufferSize)
    data = ser.read(inputBufferSize)
    out =[(int(data[i])) for i in range(0,len(data))]
    return out

def process_data(data):
    data_in = np.array(data)
    result = []
    i = 1
    while i < len(data_in)-1:
        if data_in[i] > 127:
            # Found beginning of frame
            # Extract one sample from 2 bytes
            intout = (np.bitwise_and(data_in[i],127))*128
            i = i + 1
            intout = intout + data_in[i]
            result = np.append(result,intout)
        i=i+1
    return result

######### define FFT function first ###########

def process_gaussian_fft(t,data_t,sigma_gauss):
    nfft = len(data_t) # number of points
    dt = t[1]-t[0]  # time interval
    maxf = 1/dt     # maximum frequency
    df = 1/np.max(t)   # frequency interval
    f_fft = np.arange(-maxf/2,maxf/2+df,df)          # define frequency domain

    ## DO FFT
    data_f = np.fft.fftshift(np.fft.fft(data_t)) # FFT of data

    ## GAUSSIAN FILTER
#    sigma_gauss = 25  # width of gaussian - defined in the function
    gauss_filter = np.exp(-(f_fft)**2/sigma_gauss**2)   # gaussian filter used
    data_f_filtered= data_f*gauss_filter    # gaussian filter spectrum in frquency domain
    data_t_filtered = np.fft.ifft(np.fft.ifftshift(data_f_filtered))    # bring filtered signal in time domain
    return data_t_filtered

# Feature extraction

def extract_features(data):

    # "data" needs to already be normalized
    sorted_indices = np.argsort(data)
    index_pos = sorted_indices[-1:]

    negData = -data
    sorted_indices = np.argsort(negData)
    index_neg = sorted_indices[-1:]

    index1 = min(index_pos[0], index_neg[0])
    index2 = max(index_pos[0], index_neg[0])

    peaksIndexes = [index1, index2]

    first_peak = data[index1]
    second_peak = data[index2]

    # Need a more reliable way to find the first peak. We can't always assume it's the first of second highest peak.

    # Feature 1: Is the first peak positive?
    is_first_peak_positive = first_peak > 0

    # absData = np.abs(data)
    
    # # Calculate prominences for feature extraction
    # prominences = peak_prominences(absData, peaksIndexes)[0]

    # # Feature 2: Sharpness of the first peak
    # sharpness_first_peak = prominences[0]
    
    # # Feature 3: Sharpness of the second peak
    # sharpness_second_peak = prominences[1]
    
    # Feature 4: Distance between the two peaks
    # distance_between_peaks = np.abs(first_peak - second_peak)
    
    return np.array([is_first_peak_positive])


def read_features_from_folder(folder_path, number_of_features):
    feature_matrix = np.empty((0, number_of_features))  # Initialize an empty matrix to store features

    # List all files in the directory
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            file_path = os.path.join(folder_path, filename)
            # Read the waveform data from each text file
            data = np.loadtxt(file_path)


            # Extract features from the data
            # features = extract_features(data)
            features = calculate_means(data, number_of_features)
            # Append features as a new row to the feature matrix
            feature_matrix = np.vstack((feature_matrix, features))

    return feature_matrix


def calculate_means(array, num_subsets):
    # Calculate the size of each subset
    n = len(array)
    subset_size = n // num_subsets

    # Initialize an empty list to store the means
    means = []

    # Split the array into subsets and calculate the mean of each
    for i in range(num_subsets):
        start_index = i * subset_size
        # Handle the last subset potentially having more elements
        if i == num_subsets - 1:
            subset = array[start_index:]  # Take everything remaining
        else:
            subset = array[start_index:start_index + subset_size]
        means.append(np.mean(subset))
    
    return np.array(means)

In [5]:
from pynput.keyboard import Controller, Key
import time

keyboard = Controller()

# Function to simulate key press
def simulate_key_press(key):
    keyboard.press(key)
    time.sleep(0.2)  # This is to ensure that the keypress is registered 
    keyboard.release(key)

def press_two_keys(key1, key2):
    """
    Presses two keys simultaneously.
    
    Args:
    key1: The first key to press. Can be a character like 'a' or a special key like Key.ctrl.
    key2: The second key to press. Can be a character like 'b' or a special key like Key.alt.
    """
    keyboard = Controller()
    
    # Press both keys
    keyboard.press(key1)
    keyboard.press(key2)

    # Release both keys
    keyboard.release(key1)
    keyboard.release(key2)

# Read the file and press keys based on specific words
def read_and_press_keys(label):
    if label == "pause": 
        simulate_key_press(Key.space)  # Simulate space bar press
        print("Pause")
    elif label == "right":
        press_two_keys(Key.cmd, Key.right)  # Simulate F9 key press
        print("Skip song")
    elif label == "left":
        press_two_keys(Key.cmd, Key.left)  # Simulate F8 key press 
        print("Previous song") 


In [ ]:
# Read data
baudrate = 230400
# cport = 'COM9'  # set the correct port before you run it
# cport = '/dev/tty.usbmodem141101'  # set the correct port before run it
cport = "/dev/cu.usbserial-DJ00E2W2"
ser = serial.Serial(port=cport, baudrate=baudrate)    # this initializes the animated plot
# %matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt

fig = plt.figure()
ax = fig.add_subplot(111)
plt.ion()

fig.show()
fig.canvas.draw()


In [ ]:
# Random forest training

import pandas as pd
import numpy as np
import os

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

# reading in all the eye movement files
# storing them as a dataframe with two columns: label and data
# label is the eye movement type (ex: up)
# data is a list of the numbers that were within the file 

# whoever is running the code will need to change this 

main_folder = '/Users/jameslocke/Documents/SpikerStream-master-2/Model training'


data = []


# iterating over each folder in the main folder

for folder_name in os.listdir(main_folder):
    folder_path = os.path.join(main_folder, folder_name)
    
    # if the item is a folder
    
    if os.path.isdir(folder_path):
        
        # iterate over each file in that folder 
        
        for file_name in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file_name)
            
            # if the item is a file 
            
            if os.path.isfile(file_path):
                
                # store the file content as a list 
                
                try:
                    with open(file_path, 'r', encoding='utf-8') as file:
                        numbers = file.read().splitlines()
                except UnicodeDecodeError:
                    continue
                
                # creating a dictionary 
                # with the folder name as the label and file contents as the data 
                
                data.append({'Label': folder_name, 'Data': numbers})

# making into a dataframe 

df = pd.DataFrame(data)

# checking the output 

print(df.head())
print()
print(df['Label'])

def train_random_forest(dataframe):

    X = pd.DataFrame(dataframe['Data'].values.tolist())
    print(X)
    y = dataframe['Label']
    
    # handling missing 
    imputer = SimpleImputer(strategy='mean')
    X = imputer.fit_transform(X)
    
    # splitting the data into training and testing sets 
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)
    
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    clf = RandomForestClassifier()
    
    clf.fit(X_train, y_train)
    
    y_pred = clf.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy:", accuracy)
    
    return clf

trained_classifier = train_random_forest(df)



In [311]:
# Random Forest classifier

def classify_eye_movement(file_path, classifier):

    # reading in the file 
    
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            numbers = file.read().splitlines()
    except UnicodeDecodeError:
        print("Unable to read the file due to encoding issues.")
        return None
    
    # preprocessing the data to match the format expected by the classifier
    
    # converting the numbers to float values 
    
    try:
        numbers = [float(num) for num in numbers]
    except ValueError:
        print("Error converting string data to float values.")
        return None
    
    # want the file to have the same number of features that the classifier was trained on
    # if the number of features is less than expected, pad with zeros 
    
    if len(numbers) <= 29997:
        numbers += [0] * (29997 - len(numbers))
    
    # also address if the features is more than expected
    
    elif len(numbers) > 29997:
        numbers = numbers[:29997]
    
    # converting to a numpy array with a single row 
    
    X = np.array([numbers])
    
    predicted_label = classifier.predict(X)[0]
    
    return predicted_label

# practicing

# print(classify_eye_movement('/Users/sfoulsham/Desktop/data3888/testing_examples/r1.txt', 
#                             trained_classifier))


In [ ]:
# Neural network

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, Dropout

np.random.seed(42)
tf.random.set_seed(42)

def build_model(input_dim):
    # Create a more complex neural network model
    model = Sequential([
        Dense(64, activation='relu', input_dim=input_dim),
        # Dropout(0.5),  # Adding dropout regularization
        # Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        # Dropout(0.5),  # Adding dropout regularization
        Dense(16, activation='relu'),
        # Dense(8, activation='relu'),
        Dense(3, activation='sigmoid')  # Adjust the number of output units to match the number of classes
    ])
    
    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.001),
                #   loss = 'binary_crossentropy',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model


# Example of creating a model
input_dim = 100  # Adjust this based on the actual number of features you plan to use
model = build_model(input_dim)

# Specify the path to your folder containing class 1 examples
folder_path1 = '/Users/jameslocke/Documents/SpikerStream-master-2/Model training/LookLeft'
folder_path2 = '/Users/jameslocke/Documents/SpikerStream-master-2/Model training/LookRight'
folder_path3 = '/Users/jameslocke/Documents/SpikerStream-master-2/Model training/Blink'


# Read features into a matrix
# num_labels_class1 = 31
# num_labels_class2 = 26

X_class1 = read_features_from_folder(folder_path1, input_dim)
X_class2 = read_features_from_folder(folder_path2, input_dim)
X_class3 = read_features_from_folder(folder_path3, input_dim)


# Create label arrays for class 1 and class 2. 0 for look left, 1 for look right
y_class1 = np.zeros((round(np.size(X_class1)/input_dim), 1))
y_class2 = np.ones((round(np.size(X_class2)/input_dim), 1))
y_class3 = 2*np.ones((round(np.size(X_class3)/input_dim), 1))


# Concatenate  arrays for class 1 and class 2
X = np.concatenate((X_class1, X_class2, X_class3), axis=0)
Y = np.concatenate((y_class1, y_class2, y_class3), axis=0)


# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

# Train the model
model.fit(X_train, y_train, epochs=200, batch_size=64, validation_data=(X_test, y_test))

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy*100:.2f}%")


In [ ]:
# using the neural network

folder_path = '/Users/jameslocke/Documents/SpikerStream-master-2/Model Practice/dartRight'
features = read_features_from_folder(folder_path, 100)

predictions = model.predict(features)
binary_predictions = (predictions > 0.5).astype(int)
predicted_classes = np.argmax(predictions, axis=1)

print(predicted_classes)


In [ ]:
# Calibration

import numpy as np

# Take 10-20 seconds of static data and send to the "noise" folder:

path = '/Users/jameslocke/Documents/SpikerStream-master-2/Week 10 data/noise.txt'

# To find the mean of the noise, we use:

data = np.loadtxt(path) # numpy array of calibration data

# Calculate reduced data where deviations from the mean are less than one standard deviation
mean_data = np.mean(data)
std_data = np.std(data)
reduced = data[np.abs(data - mean_data) < 1 * std_data]

# Calculate the mean of the reduced data
redAve = np.mean(reduced)

# The mean of the noise - to be used when centering event waveforms around 0
noiseMean = redAve

print(noiseMean)


In [ ]:
# take continuous data stream 
inputBufferSize = 2000 # keep betweein 2000-20000
ser.timeout = inputBufferSize/20000.0  # set read timeout, 20000 is one second
# ser.set_buffer_size(rx_size = inputBufferSize)

%matplotlib inline

total_time = 15.0; # time in seconds [[1 s = 20000 buffer size]]
max_time = 15; # time plotted in window [s]
N_loops = 20000.0/inputBufferSize*total_time

T_acquire = inputBufferSize/20000.0    # length of time that data is acquired for 
N_max_loops = max_time/T_acquire    # total number of loops to cover desire time window

fig = plt.figure()
ax1 = fig.add_subplot(1,1,1)
plt.ion()
# fig.show()
fig.canvas.draw()

# event detection variables

event = 0  # 1 if an event is happening at this time step.
events = []
event_count = 0

event_ended = 0


Sd = []
Ave = []

upThreshold = 10
downThreshold = 2
gone_up = False
just_under = False


from IPython.display import clear_output, display

for k in range(0,int(N_loops)):
    data = read_arduino(ser,inputBufferSize)
    data_temp = process_data(data)
    
    # DO THE GAUSSIAN FILTERING FIRST - OTHERWISE YOUR CODE SLOWS DOWN
    # define temporary time array OF THE WINDOW
    T = inputBufferSize/20000.0*np.linspace(0,1,(data_temp).size)
    sigma_gauss = 25
    data_temp_filtered = process_gaussian_fft(T,data_temp,sigma_gauss)

    if k <= N_max_loops:
        if k==0:
            data_plot = data_temp
            data_plot_filtered = data_temp_filtered
        else:
            data_plot = np.append(data_temp,data_plot)
            data_plot_filtered = np.append(data_temp_filtered,data_plot_filtered)

        t = (min(k+1,N_max_loops))*inputBufferSize/20000.0*np.linspace(0,1,(data_plot).size)
    else:
        data_plot = np.roll(data_plot,len(data_temp))
        data_plot[0:len(data_temp)] = data_temp
        
        data_plot_filtered = np.roll(data_plot_filtered,len(data_temp_filtered))
        data_plot_filtered[0:len(data_temp_filtered)] = data_temp_filtered
        
    t = (min(k+1,N_max_loops))*inputBufferSize/20000.0*np.linspace(0,1,(data_plot).size)
    t_filtered = (min(k+1,N_max_loops))*inputBufferSize/20000.0*np.linspace(0,1,(data_plot_filtered).size)
        
    t_filtered = (min(k+1,N_max_loops))*inputBufferSize/20000.0*np.linspace(0,1,(data_plot_filtered).size)

    # same thing here
    
    ax1.clear()
    ax1.set_xlim(0, max_time)
    plt.xlabel('time [s]')
    ax1.plot(t_filtered,data_plot_filtered)

    clear_output(wait=True)
    display(fig)

    # event detection

    data = np.abs(data_temp_filtered)

    sd = np.std(data)
    ave = np.mean(data)
    Sd.append(sd)
    Ave.append(ave)

    if event == 0:
        if sd > upThreshold:  # new event
            event_count += 1
            event = 1
            new_event = data
            events.append(new_event)
            print("new event")
    else:
        current_event = events[event_count - 1]
        current_event = np.concatenate((current_event, data))
        events[event_count - 1] = current_event

        # event may have ended if it returns close to mean
        # wait until SD turns around until we declare the event to be ended
        if sd <= downThreshold:
            if not just_under:
                just_under = True
                under_count = 1
            else:
                under_count += 1
                if under_count >= 1:
                    event = 0
                    event_ended = 1
                    print('End')

                    current_event = events[event_count - 1]
                    normalised_event = current_event - noiseMean

                    features = extract_features(normalised_event)

                    prediction = model.predict(features)
                    binary_prediction = (prediction > 0.5).astype(int)

                    print(binary_prediction)

                    if binary_prediction[0][0] == 0:
                        print('Left')
                        label = 'left'
                    elif binary_prediction[0][0] == 1:
                        print('Right')
                        label = 'right'
                    elif binary_prediction[0][0] == 2:
                        print('blink')
                        label = 'pause'
                    
                    read_and_press_keys(label)

                    # Random forest
                    # eventFilePath = '/Users/jameslocke/Documents/SpikerStream-master-2/Week 10 data/current_event.txt' 
                    # np.savetxt(eventFilePath, normalised_event)
                    # print(classify_eye_movement(eventFilePath, trained_classifier))

                    

            if not gone_up:
                gone_up = (np.sign(sd - prev_sd) + 1) / 2  # check for turning around
            else:
                # event ends if Sd previously went up and now goes down
                event = np.sign(sd - prev_sd) == 1

                event_ended = 1
                print('End')


                current_event = events[event_count - 1]
                normalised_event = current_event - noiseMean
                # normalised_event = current_event

                # Neural network

                features = extract_features(normalised_event)

                prediction = model.predict(features)
                binary_prediction = (prediction > 0.5).astype(int)

                print(binary_prediction)

                if binary_prediction[0][0] == 0:
                    print('Left')
                    label = 'left'
                elif binary_prediction[0][0] == 1:
                    print('Right')
                    label = 'right'
                elif binary_prediction[0][0] == 2:
                    print('blink')
                    label = 'pause'
                
                read_and_press_keys(label)

                # Random forest

                # eventFilePath = '/Users/jameslocke/Documents/SpikerStream-master-2/Week 10 data/current_event.txt' 
                # np.savetxt(eventFilePath, normalised_event)
                # print(classify_eye_movement(eventFilePath, trained_classifier))
            


                # classify event. Algorithm below:

                # check if the first peak is positive or negative. If it's negative, we know it's a look left, or non event.

                # If it's positive, 

            if not event:
                gone_up = False
                just_under = False
        else:
            gone_up = False
            just_under = False

    prev_ave = ave
    prev_sd = sd

    if event_ended:
        current_event = events[event_count - 1]
        event_ended = 0



    



    # if an event ended at this timestep, it would now be fed to the classifier. 
    # Before doing this, we would also normalise the data, and check the event's amplitude.
    # If the amplitude is too small, we could say that it's no longer an event.
    # Idea: if event's maximum value isn't more that 2 SD's away from the event's mean, then it's no longer an event.

# Need to save t_filtered as well. Also the FFT spectrum for an event.


absArray = np.abs(data_plot_filtered)

np.savetxt('/Users/jameslocke/Documents/SpikerStream-master-2/Week 7 data/LookLeft/raw_data.txt', data_plot)
np.savetxt('/Users/jameslocke/Documents/SpikerStream-master-2/Week 7 data/LookLeft/filtered_data.txt', data_plot_filtered)
np.savetxt('/Users/jameslocke/Documents/SpikerStream-master-2/Week 7 data/LookLeft/abs_filtered_data.txt', absArray)

i = 0
while i < event_count:
    currentEvent = events[i]
    path = '/Users/jameslocke/Documents/SpikerStream-master-2/Week 10 data/event_number' + str(i) + '.txt'
    np.savetxt(path, currentEvent)
    i += 1

path = '/Users/jameslocke/Documents/SpikerStream-master-2/Week 10 data/noise.txt'
np.savetxt(path, absArray)



In [ ]:
path = '/Users/jameslocke/Documents/SpikerStream-master-2/Week 10 data/noise.txt'
np.savetxt(path, absArray)

In [325]:
path = '/Users/jameslocke/Documents/SpikerStream-master-2/Week 10 data/SD.txt'
np.savetxt(path, Sd)

In [ ]:
def freq_fft(t,data_t):
    nfft = len(data_t) # number of points
    dt = t[1]-t[0]  # time interval
    maxf = 1/dt     # maximum frequency
    df = 1/np.max(t)   # frequency interval
    f_fft = np.arange(-maxf/2,maxf/2+df,df)          # define frequency domain

    ## DO FFT
    data_f = np.fft.fftshift(np.fft.fft(data_t)) # FFT of data

    return f_fft, data_f

In [ ]:
# close serial port if necessary
if ser.read():
    ser.flushInput()
    ser.flushOutput()
    ser.close()

In [ ]:
path = '/Users/jameslocke/Documents/SpikerStream-master-2/Week 9 data/SD.txt'
np.savetxt(path, Sd)

In [ ]:
# Alternative event detection. Doesn't use the turning point or require SD to hang below threshold for a period of time. 
# More suitable for larger buffer size.


# take continuous data stream 
inputBufferSize = 4000 # keep betweein 2000-20000
ser.timeout = inputBufferSize/20000.0  # set read timeout, 20000 is one second
# ser.set_buffer_size(rx_size = inputBufferSize)

%matplotlib inline

total_time = 15.0; # time in seconds [[1 s = 20000 buffer size]]
max_time = 15; # time plotted in window [s]
N_loops = 20000.0/inputBufferSize*total_time

T_acquire = inputBufferSize/20000.0    # length of time that data is acquired for 
N_max_loops = max_time/T_acquire    # total number of loops to cover desire time window

fig = plt.figure()
ax1 = fig.add_subplot(1,1,1)
plt.ion()
# fig.show()
fig.canvas.draw()

# event detection variables

event = 0  # 1 if an event is happening at this time step.
events = []
event_count = 0

event_ended = 0


Sd = []
Ave = []

upThreshold = 10
downThreshold = 5
gone_up = False
just_under = False


from IPython.display import clear_output, display

for k in range(0,int(N_loops)):
    data = read_arduino(ser,inputBufferSize)
    data_temp = process_data(data)
    
    # DO THE GAUSSIAN FILTERING FIRST - OTHERWISE YOUR CODE SLOWS DOWN
    # define temporary time array OF THE WINDOW
    T = inputBufferSize/20000.0*np.linspace(0,1,(data_temp).size)
    sigma_gauss = 25
    data_temp_filtered = process_gaussian_fft(T,data_temp,sigma_gauss)

    if k <= N_max_loops:
        if k==0:
            data_plot = data_temp
            data_plot_filtered = data_temp_filtered
        else:
            data_plot = np.append(data_temp,data_plot)
            data_plot_filtered = np.append(data_temp_filtered,data_plot_filtered)

        t = (min(k+1,N_max_loops))*inputBufferSize/20000.0*np.linspace(0,1,(data_plot).size)
    else:
        data_plot = np.roll(data_plot,len(data_temp))
        data_plot[0:len(data_temp)] = data_temp
        
        data_plot_filtered = np.roll(data_plot_filtered,len(data_temp_filtered))
        data_plot_filtered[0:len(data_temp_filtered)] = data_temp_filtered
        
    t = (min(k+1,N_max_loops))*inputBufferSize/20000.0*np.linspace(0,1,(data_plot).size)
    t_filtered = (min(k+1,N_max_loops))*inputBufferSize/20000.0*np.linspace(0,1,(data_plot_filtered).size)
        
    t_filtered = (min(k+1,N_max_loops))*inputBufferSize/20000.0*np.linspace(0,1,(data_plot_filtered).size)

    # same thing here
    
    ax1.clear()
    ax1.set_xlim(0, max_time)
    plt.xlabel('time [s]')
    ax1.plot(t_filtered,data_plot_filtered)

    clear_output(wait=True)
    display(fig)

    # event detection

    data = np.abs(data_temp_filtered)

    sd = np.std(data)
    ave = np.mean(data)
    Sd.append(sd)
    Ave.append(ave)

    if event == 0:
        if sd > upThreshold:  # new event
            event_count += 1
            event = 1
            new_event = data
            events.append(new_event)
            print("new event")
    else:
        if sd <= downThreshold:
            event = 0
            print('End')

            current_event = events[event_count - 1]
            normalised_event = current_event - noiseMean

            features = extract_features(normalised_event)

            prediction = model.predict(features)
            binary_prediction = (prediction > 0.5).astype(int)

            if binary_prediction[0][0] == 0:
                print('Left')
                label = 'left'
            elif binary_prediction[0][0] == 1:
                print('Right')
                label = 'right'
            elif binary_prediction[0][0] == 2:
                print('blink')
                label = 'pause'
            
            read_and_press_keys(label)

        else:
            current_event = events[event_count - 1]
            current_event = np.concatenate((current_event, data))
            events[event_count - 1] = current_event
                
    prev_ave = ave
    prev_sd = sd


    # if an event ended at this timestep, it would now be fed to the classifier. 
    # Before doing this, we would also normalise the data, and check the event's amplitude.
    # If the amplitude is too small, we could say that it's no longer an event.
    # Idea: if event's maximum value isn't more that 2 SD's away from the event's mean, then it's no longer an event.

# Need to save t_filtered as well. Also the FFT spectrum for an event.


absArray = np.abs(data_plot_filtered)

np.savetxt('/Users/jameslocke/Documents/SpikerStream-master-2/Week 7 data/LookLeft/raw_data.txt', data_plot)
np.savetxt('/Users/jameslocke/Documents/SpikerStream-master-2/Week 7 data/LookLeft/filtered_data.txt', data_plot_filtered)
np.savetxt('/Users/jameslocke/Documents/SpikerStream-master-2/Week 7 data/LookLeft/abs_filtered_data.txt', absArray)

i = 0
while i < event_count:
    currentEvent = events[i]
    path = '/Users/jameslocke/Documents/SpikerStream-master-2/Week 10 data/event_number' + str(i) + '.txt'
    np.savetxt(path, currentEvent)
    i += 1

path = '/Users/jameslocke/Documents/SpikerStream-master-2/Week 10 data/noise.txt'
np.savetxt(path, absArray)

